**Modelo de Aprendizado para Previsão da Resistência do Concreto - por Yuri Miguel - Engenheiro Civil**
![Concrete Civil Engineering](http://upload.wikimedia.org/wikipedia/commons/thumb/5/5b/US_Navy_090724-N-1831G-002_An_Indonesian_Navy_engineer_and_a_U.S._Navy_Seabee_pour_concrete_for_an_engineering_civic_action_program_at_Pusaka_Rakyat_Primary_School.jpg/800px-thumbnail.jpg)

Este trabalho busca apresentar de forma simples e direta um modelo para previsão do **"Fck"** ( Valor de Resistência do Concreto a Compressão), através da quantidade de material usado para sua formação, comumente conhecido com Traço do concreto. 
O Dataset utilizado é proveniente da plataforma **Kaggle**, é foi disponibilizado através do **'Machine Learning Reporitory'**:
    
> *Abstract: Concrete is the most important material in civil engineering. The concrete compressive strength is a highly nonlinear function of age and ingredients.
Original Owner and Donor 
Prof. I-Cheng Yeh 
**Department of Information Management 
Chung-Hua University, 
Hsin Chu, Taiwan 30067, R.O.C.**
e-mail:icyeh '@' chu.edu.tw 
TEL:886-3-5186511 *

Date Donated: August 3, 2007     

[Link de Acesso Kaggle](http://www.kaggle.com/maajdl/yeh-concret-data)
[Link de Acesso MLR](http://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength)

**OBS: É importante salientar que o modelo apresentado tem como objetivo apenas o estudo e a divulgação da utilização de machine learning dentro da engenharia civil, não podendo ser levado em consideração para definições técnicas ou para dimensionamento de estruturas. **

<a name='TOPO'></a>
<h1>Índice</h1>
<ol>
<li><a href='#DATACLEANING'>Limpeza de Dados (Data Cleaning)</a></li>
<li><a href='#DATAANALYSIS'>Análise dos Dados (Data Analysis)</a></li>
<li><a href='#DATATRAIN'>Separação dos Dados (Data Train)</a></li>
<li><a href='#DATAMODEL'>Modelo de Previsão (Data Model)</li>
<li><a href='#DATAAVAL'>Avaliação do Modelo (Data Evaluation)</li>
<li><a href='#CONCLUSAO'>Conclusão</li>
</ol>

<a name='DATACLEANING'></a>
<h1>Limpeza de Dados (Data Cleaning)</h1>

In [ ]:
#Importando as bibliotecas 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # Criação de gráficos. 
import seaborn as sns # Desing de gráficos mais atraentes.

from scipy.stats import norm
from scipy import stats

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics



In [ ]:
%matplotlib inline

In [ ]:
#Criando o DataFrame através do arquivo Concret_Data_Yeh
df = pd.read_csv('../input/Concrete_Data_Yeh.csv')

# Vamos Renomear as colunas com a tradução das nomenclaturas para o português.
# Todas as colunas de materiais estão na unidade de Kg/m³.
# A idade do concreto está em Dias.
# Já o Fck está em Mpa.
df.columns = ['Cimento','Escória de Alto Forno', 'Escória', 'Água', 'Superplastificante', 'Agregado Graudo', 'Agregado Fino', 'Idade do Concreto', 'Fck']
df.head(10)

In [ ]:
# Vamos Arredondar o valor do Fck
df.round({'Fck': 0})

<a name="DATAANALYSIS"></a>
<a href='#TOPO'>Voltar ao índice</a>
<h1>Análise dos Dados (Data Analysis)</h1>

In [ ]:
# Vamos agora obter uma descrição dos dados provenientes do DataFrame.
df.describe()

In [ ]:
# Vamos avaliar a tipologia dos nossos dados;
df.dtypes

A idade do concreto é outro importante fator para essa análise, pois quando precisamos encontrar um modelo preditivo para a resistência do concreto a compressão precisamos garantir que as amostras tinham idade suficiente para atingir a resistência progamada. 
Em geral a resistência do concreto não sofre grandes variações após os 28 dias de idade. Sendo esta uma boa idade para considerarmos nessa análise. 

In [ ]:
# Vamos plotar um gráfico de barras para determinar quantas unidade de corpor de prova temos 
# para cada idade do concreto;
df['Idade do Concreto'].value_counts().plot.bar()

plt.xlabel('Idade do Concreto')
plt.ylabel('Quantidade de ocorrências')

In [ ]:
# Vamos criar um novo DataFrame apenas com os corpos de prova que tiveram sua resistência medida 
# com 28 dias

df28 = df[df['Idade do Concreto'] == 28]

df28.head()

In [ ]:
# Agora vamos avaliar quantas entradas temos no nosso novo DataFrame
df28.info()

In [ ]:
# Precisamos verificar a nova média do Fck para esse novo DataFrame
# uma vez que ela apresenta um valor de parâmetro de avaliação.
df28.describe()

A função **"pairplot"** é definida por sua documentação como:

> Por padrão, essa função criará uma grade de Eixos de forma que cada variável na datavontade seja compartilhada no eixo y em uma única linha e no eixo x em uma única coluna. Os Eixos diagonais são tratados de maneira diferente, desenhando um gráfico para mostrar a distribuição univariada dos dados para a variável nessa coluna.

In [ ]:
# Vamos utilizar o PairPlot para tentar visualizar como é a distribuição do nossos dados.
sns.pairplot(df28)

Como o nosso objetivo é estimar o valor do **Fck**, devemos achar um modelo que consiga prever o valor deste parâmetro. 

Pode-se observar através da ultima linha de gráficos que o valor do **Fck**, apresenta uma leve tendência para uma regressão linear apenas quando comparado a quantidade de cimento presente no concreto. Oque faz sentido uma vez que o **cimento** é o maior responsável pelo ganho de resistência. Porém essa padrão não é observado quando comparado as outras colunas.


A função **"heatmap"**, apresenta a criação de uma mapa de calor dos nossos dados. A documentação apresenta sua função como:
> Um mapa de calor (ou mapa de calor ) é uma representação gráfica de dados em que os valores individuais contidos em uma matriz são representados como cores.

In [ ]:
sns.heatmap(df28.corr())

Outra importante avaliação é verificar se a nossa váriável escolhida como target, segue uma tendência normal, afim de facilitar a nossa predição.

In [ ]:
# Vamos Verificar se a variável de Target segue uma tendencia de distribuição normal.
sns.distplot(df28['Fck'], fit=norm);
fig = plt.figure()
res = stats.probplot(df28['Fck'], plot=plt)

<a name="DATATRAIN"></a>
<a href='#TOPO'>Voltar ao índice</a>
<h1>Separação dos Dados (Data Train)</h1>

In [ ]:
# Vamos começar dividindo os nossos dados em atributos e rótulos

#Conjunto de todas as colunas do DataFrame,menos as colunas 'Fck' e 'Idade do Concreto'
X = df28.drop(['Fck', 'Idade do Concreto'], axis=1) 

#Conjunto apenas com a coluna 'Fck', variável que queremos prever
y = df28['Fck'] 

In [ ]:
# Agora vamos dividir os nossos dados em conjuntos de treinamento e teste.
# Para esse exemplo vamos utilizar o método 'train_test_split', que fará a divisão automática dos dados.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

<a name="DATAMODEL"></a>
<a href='#TOPO'>Voltar ao índice</a>
<h1>Modelo de Previsão (Data Model)</h1>

Vamos tentar inicialmente usar um modelo de árvore de regressão para prever o valor do fck:

Para isso vamos utilizar uma classe do **'sklearn'** chamada **'RandomForestRegressor'**

In [ ]:
# Primeiramente vamos instanciar a classe 'DecisionForestRegressor' e chamar o método 'fit'
rf = RandomForestRegressor (n_estimators=1000, random_state=42)

rf.fit(X_train, y_train)

In [ ]:
# Para fazer previsões no conjunto de teste vamos utilizar o método 'predict'
y_pred = rf.predict(X_test)

<a name="DATAAVAL"></a>
<a href='#TOPO'>Voltar ao índice</a>
<h1>Avaliação do Modelo (Data Evaluation)</h1>

In [ ]:
# Agora vamos comparar alguns dos nossos valores previstos com os valores reais e ver o quão preciso é o nosso modelo.
df_new = pd.DataFrame({'Valor_Real':y_test, 'Valor_Previsto':y_pred})
df_new

Alguns valores chegaram bem proximo dos valores reiais, porém outros valores ficaram com um erro muito alto para o que é esperado.
Isso era esperado uma vez que o 'train_test_split', dívide os dados aleatoriamente.

Para avaliar o desempenho do algoritmo de regressão, as métricas comumente utilizadas são do **'erro médio absoluto'** , **'erro quadrático médio'** , e **'raiz do erro quadrático médio'** . 
A biblioteca **Scikit-Learn** contém funções que podem ajudar a calcular esses valores para nós. Para fazer isso, use este código do **'metrics'** pacote:

In [ ]:
# Avaliando o modelo através do pacote 'metrics'.
print ('Erro Médio Absoluto:', metrics.mean_absolute_error(y_test, y_pred))
print ('Erro Quadrático Médio:', metrics.mean_squared_error(y_test, y_pred))
print ('Raiz do Erro Quadrático Médio:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
# Calculando o erro médio absoluto por outro meio
errors = abs(y_pred - y_test)

# Aqui calculamos o percentual do erro médio absoluto (MAPE)
mape =100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print ("Acurácia:", round(accuracy, 2), "%.")


<a name='CONCLUSAO'></a>
<a href='#TOPO'>Voltar ao índice</a>
<h1>Conclusão</h1>

Em breve postarei um nova tentativa em diminuir esse erro.

Caso tenha interesse, você pode me seguir nas redes sociais e enviar críticas e sugestões. 

<img src = "https://cdn-images-1.medium.com/max/1125/1*dDNpLKu_oTLzStsDTnkJ-g.png" width=75 align=left><a href='https://github.com/13yurimiguel'>13yurimiguel</a><p>

<img src = "https://upload.wikimedia.org/wikipedia/commons/7/7c/Kaggle_logo.png" width=70 align=left><a href='https://www.kaggle.com/yurimiguel'>yurimiguel</a><p>
    
<img src = "http://s.glbimg.com/po/tt/f/original/2011/05/18/linkedin_logo_1.jpg" width=70 align=left><a href='https://www.linkedin.com/in/yuri-miguel-23295b87/'>Yuri Miguel</a><p>
    
<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/2/2a/Instagram_logo.svg/1200px-Instagram_logo.svg.png" width=75 align=left><a href='https://www.instagram.com/y.m.eng/'>@y.m.eng</a><p>
